# BYOC training for paddleOCR

In [17]:
!cat Dockerfile

ARG REGISTRY_URI
FROM ${REGISTRY_URI}/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04

RUN mkdir -p /opt/ml/model

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/ml/code:${PATH}"
ENV PATH="/opt/program:${PATH}"

##########################################################################################
# SageMaker requirements
##########################################################################################
## install flask
RUN pip install networkx==2.3 flask gevent gunicorn boto3 transformers==4.6.0 datasets==1.11.0 sentencepiece==0.1.91 pytorch_lightning==0.8.1 jieba editdistance -i https://opentuna.cn/pypi/web/simple
### Install nginx notebook
RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# forward request and error logs to docker log collector
RUN ln -sf /dev/stdout /var/log/nginx/access.log
RUN ln -sf /dev/stderr /var/log/ngin

In [18]:
#first build docker
!sh build_and_push.sh absa-train

set -e
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
image=$1

if [ "$image" == "" ]
then
    echo "Use image name absa"
    image="absa"
fi

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi

# Get the region defined in the current configuration
region=$(aws configure get region)
#regions=$(aws ec2 describe-regions --all-regions --query "Regions[].{Name:RegionName}" --output text)

#for region in $regions; do

#aws s3 cp s3://aws-solutions-${region}/spot-bot-models/cars/model.tar.gz ./
#tar zxvf model.tar.gz
# TODO: update regional location based on https://amazonaws-china.com/releasenotes/available-deep-learning-containers-

## step1: upload data

In [19]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role

#sess = sage.Session()
sess = sage.LocalSession()

WORK_DIRECTORY = "../data/tasd/shulex"

# S3 prefix
prefix = "shulex-datalab"

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [20]:
data_location

's3://sagemaker-us-west-2-847380964353/shulex-datalab'

In [16]:

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "847380964353.dkr.ecr.us-west-2.amazonaws.com/absa-train"

sess = sage.LocalSession()

hyperparameters = {
    "task" : "tasd", 
    #"data_root" : "/opt/ml/input/data/training",
    "dataset" : "shulex", 
    "model_name_or_path" : "t5-base", 
    "paradigm": "extraction",
    "gradient_accumulation_steps": "2",
    "eval_batch_size" :"16",
    "train_batch_size" :"2",
    "learning_rate" :"3e-4",
    "num_train_epochs":"1",
    "do_train":True,
    "do_eval":False,
    "do_direct_eval":False
    #"out_dir":"/opt/ml/output",
}

train = sage.estimator.Estimator(
    image,
    role,
    sagemaker_session=sess,
    instance_type='local_gpu',
    instance_count=1,
    hyperparameters=hyperparameters,
)


train.fit(data_location)

Creating r3jr6r1fyy-algo-1-yq2ya ... 
Creating r3jr6r1fyy-algo-1-yq2ya ... done
Attaching to r3jr6r1fyy-algo-1-yq2ya
r3jr6r1fyy-algo-1-yq2ya | sed: can't read changehostname.c: No such file or directory
r3jr6r1fyy-algo-1-yq2ya | gcc: error: changehostname.c: No such file or directory
r3jr6r1fyy-algo-1-yq2ya | gcc: fatal error: no input files
r3jr6r1fyy-algo-1-yq2ya | compilation terminated.
r3jr6r1fyy-algo-1-yq2ya | gcc: error: changehostname.o: No such file or directory
r3jr6r1fyy-algo-1-yq2ya | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
r3jr6r1fyy-algo-1-yq2ya | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
r3jr6r1fyy-algo-1-yq2ya | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
r3jr6r1fyy-algo-1-yq2ya | ERROR: ld.so: object '/libchangehostname.so' fro

Failed to delete: /tmp/tmpza6lhq2t/algo-1-yq2ya Please remove it manually.


KeyboardInterrupt: 

# train

In [ ]:

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "847380964353.dkr.ecr.us-west-2.amazonaws.com/absa"

sess = sage.Session()
hyperparameters = {
    "task" : "tasd", 
    #"data_root" : "/opt/ml/input/data/training",
    "dataset" : "shulex", 
    "model_name_or_path" : "t5-base", 
    "paradigm": "extraction",
    "gradient_accumulation_steps": "2",
    "eval_batch_size" :"16",
    "train_batch_size" :"2",
    "learning_rate" :"3e-4",
    "num_train_epochs":"1",
    "do_train":True,
    "do_eval":False,
    "do_direct_eval":False
    #"out_dir":"/opt/ml/output",
}

train = sage.estimator.Estimator(
    image,
    role,
    instance_count = 1,
    sagemaker_session=sess,
    instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters,
)


train.fit(data_location)

2022-05-13 05:20:50 Starting - Starting the training job...
2022-05-13 05:21:16 Starting - Preparing the instances for trainingProfilerReport-1652419250: InProgress
.......

# Clean up
When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [27]:
print(endpoint_name)
sage.delete_endpoint(EndpointName=endpoint_name)
sage.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sage.delete_model(ModelName=model_name)

paddle-ep--2022-04-13-05-54-10


{'ResponseMetadata': {'RequestId': '842e4591-7ecf-42bb-950e-8d9cb273a478',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '842e4591-7ecf-42bb-950e-8d9cb273a478',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 13 Apr 2022 06:02:27 GMT'},
  'RetryAttempts': 0}}

In [24]:
endpoint_name

'paddle-ep--2022-04-13-05-54-10'